<a href="https://colab.research.google.com/github/rodrigozago/compiladores/blob/master/Lexico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tabulate import tabulate

Analisador léxico
==================

Definição da tabela de simbolos
--------------------

Classe que define a tabela de simbolos.

symTable: Estrutura do tipo dict para armazenar os tokens:

```{ lexema<string> : token<Token> }```

In [2]:
class SymTable:
    def __init__(self):
        self.symTable = dict();
    def addNewEntry(self, token):
        if(token.lexema not in self.symTable):
            self.symTable[token.lexema] = token
        return token
    def getEntry(self, lexema):
        if(lexema in self.symTable):
            return self.symTable[lexema]
        else:
            return None

Definição da classe Token
--------------------------

In [3]:
class Token:
    def __init__(self, lexema, token, tipo):
        self.lexema = lexema
        self.token = token
        self.tipo = tipo
    
    def __str__(self):
        return "Lexema: {} Token: {} Tipo: {}".format(self.lexema, self.token, self.tipo)

    def getList(self):
        return [self.lexema, self.token, self.tipo]

Adicionando palavras chave da linguagem na tabela de simbolos
---------------------------------------

In [4]:
# Creating symbol table
table = SymTable()

# Add keywords into symbol table
try:
    table.addNewEntry(Token('inicio', 'inicio', '-'))
    table.addNewEntry(Token('varinicio', 'varinicio', '-'))
    table.addNewEntry(Token('varfim', 'varfim', '-'))
    table.addNewEntry(Token('escreva', 'escreva', '-'))
    table.addNewEntry(Token('leia', 'leia', '-'))
    table.addNewEntry(Token('se', 'se', '-'))
    table.addNewEntry(Token('entao', 'entao', '-'))
    table.addNewEntry(Token('fimse', 'fimse', '-'))
    table.addNewEntry(Token('fim', 'fim', '-'))
    table.addNewEntry(Token('inteiro', 'inteiro', '-'))
    table.addNewEntry(Token('lit', 'lit', '-'))
    table.addNewEntry(Token('real', 'real', '-'))    
except Exception as e:
    print("erro: {}".format(e))



In [5]:
print(table.addNewEntry(Token('inicio', 'inicio', '-')))

Lexema: inicio Token: inicio Tipo: -


In [6]:
print(table.symTable['inicio'])

Lexema: inicio Token: inicio Tipo: -


In [19]:
table.symTable

{'inicio': <__main__.Token at 0x7f7008cc92e8>,
 'varinicio': <__main__.Token at 0x7f7008cc9390>,
 'varfim': <__main__.Token at 0x7f7008cc93c8>,
 'escreva': <__main__.Token at 0x7f7008cc9400>,
 'leia': <__main__.Token at 0x7f7008cc9438>,
 'se': <__main__.Token at 0x7f7008cc9470>,
 'entao': <__main__.Token at 0x7f7008cc94a8>,
 'fimse': <__main__.Token at 0x7f7008cc94e0>,
 'fim': <__main__.Token at 0x7f7008cc9518>,
 'inteiro': <__main__.Token at 0x7f7008cc9550>,
 'lit': <__main__.Token at 0x7f7008cc9588>,
 'real': <__main__.Token at 0x7f7008cc95c0>,
 'A': <__main__.Token at 0x7f7008cde6d8>,
 'B': <__main__.Token at 0x7f7008cde320>,
 'D': <__main__.Token at 0x7f7008cde8d0>,
 'C': <__main__.Token at 0x7f7008cde588>,
 'n': <__main__.Token at 0x7f7008cc9e10>}

Tabela dos estados finais
-------------

In [9]:
state_tokens = {
    1: 'Num',
    2: 'Num',
    3: 'Num',
    6: 'Num',
    8: 'Literal',
    9: 'id',
    11: 'Comentário',
    12: 'EOF',
    13: 'OPR',
    14: 'OPR',
    15: 'OPR',
    16: 'OPR',
    17: 'RCB',
    18: 'OPR',
    19: 'OPM',
    20: 'AB_P',
    21: 'FC_P',
    22: 'PTV'
}

Definição do autômato finito
-----------------------------

Definicao da class DFA

In [10]:
class DFA:
    current_state = None;
    def __init__(self, states, alphabet, transition_function, start_state, accept_states):
        self.states = states;
        self.alphabet = alphabet;
        self.transition_function = transition_function;
        self.start_state = start_state;
        self.accept_states = accept_states;
        self.current_state = start_state;
        return;
    
    def transition_to_state_with_input(self, input_value):
        if((self.current_state, input_value) not in self.transition_function.keys()):
            if((self.current_state, '@') in self.transition_function.keys()):
                self.current_state = self.transition_function[(self.current_state, '@')];
                return True;  
            return False;
        previous_state = self.current_state;
        self.current_state = self.transition_function[(self.current_state, input_value)];
        # debug do automato
        # print("({}, {}) -> {}".format(previous_state, input_value, self.current_state))
        return True;
            
            
    def addToSymbolTable(self, lexema, token):
        result = table.getEntry(lexema)
        if result:
            return result
        else: 
            return table.addNewEntry(token)
        
    
    def in_accept_state(self):
        return self.current_state in accept_states;
    
    def go_to_initial_state(self):
        self.current_state = self.start_state;
        return;

Definindo função de transição

In [11]:
tf = dict();

# Begin tf[(0, D)] = 1;
tf[(0, '0')] = 1;
tf[(0, '1')] = 1;
tf[(0, '2')] = 1;
tf[(0, '3')] = 1;
tf[(0, '4')] = 1;
tf[(0, '5')] = 1;
tf[(0, '6')] = 1;
tf[(0, '7')] = 1;
tf[(0, '8')] = 1;
tf[(0, '9')] = 1;
# End tf[(0, D)] = 1;

# Begin tf[(1, D)] = 1;
tf[(1, '0')] = 1;
tf[(1, '1')] = 1;
tf[(1, '2')] = 1;
tf[(1, '3')] = 1;
tf[(1, '4')] = 1;
tf[(1, '5')] = 1;
tf[(1, '6')] = 1;
tf[(1, '7')] = 1;
tf[(1, '8')] = 1;
tf[(1, '9')] = 1;
# End tf[(1, D)] = 1;

#TODO RESOLVER \. 
tf[(1, '.')] = 2;
tf[(1, 'E')] = 4;
tf[(1, 'e')] = 4;

# Begin tf[(2, D)] = 3;
tf[(2, '0')] = 3;
tf[(2, '1')] = 3;
tf[(2, '2')] = 3;
tf[(2, '3')] = 3;
tf[(2, '4')] = 3;
tf[(2, '5')] = 3;
tf[(2, '6')] = 3;
tf[(2, '7')] = 3;
tf[(2, '8')] = 3;
tf[(2, '9')] = 3;
# End tf[(2, D)] = 3;

# TODO VERIFICAR COM O PRATES
# tf[(2, "QUALQUER COISA")] = 23;

# Begin tf[(3, D)] = 3;
tf[(3, '0')] = 3;
tf[(3, '1')] = 3;
tf[(3, '2')] = 3;
tf[(3, '3')] = 3;
tf[(3, '4')] = 3;
tf[(3, '5')] = 3;
tf[(3, '6')] = 3;
tf[(3, '7')] = 3;
tf[(3, '8')] = 3;
tf[(3, '9')] = 3;
# End tf[(3, D)] = 3;

tf[(3, 'E')] = 4;
tf[(3, 'e')] = 4;
tf[(4, '+')] = 5;
tf[(4, '-')] = 5;

# Begin tf[(4, D)] = 6;
tf[(4, '0')] = 6;
tf[(4, '1')] = 6;
tf[(4, '2')] = 6;
tf[(4, '3')] = 6;
tf[(4, '4')] = 6;
tf[(4, '5')] = 6;
tf[(4, '6')] = 6;
tf[(4, '7')] = 6;
tf[(4, '8')] = 6;
tf[(4, '9')] = 6;
# End tf[(4, D)] = 6;

# TODO VERIFICAR COM O PRATES (ACONTECEU ERRO?)
# tf[(4, 'QUALQUER COISA')] = 23;

# Begin tf[(5, D)] = 6;
tf[(5, '0')] = 6;
tf[(5, '1')] = 6;
tf[(5, '2')] = 6;
tf[(5, '3')] = 6;
tf[(5, '4')] = 6;
tf[(5, '5')] = 6;
tf[(5, '6')] = 6;
tf[(5, '7')] = 6;
tf[(5, '8')] = 6;
tf[(5, '9')] = 6;
# End tf[(5, D)] = 6;

# TODO VERIFICAR COM O PRATES (ACONTECEU ERRO?)
# tf[(5, 'QUALQUER COISA')] = 23;

# Begin tf[(6, D)] = 6;
tf[(6, '0')] = 6;
tf[(6, '1')] = 6;
tf[(6, '2')] = 6;
tf[(6, '3')] = 6;
tf[(6, '4')] = 6;
tf[(6, '5')] = 6;
tf[(6, '6')] = 6;
tf[(6, '7')] = 6;
tf[(6, '8')] = 6;
tf[(6, '9')] = 6;
# End tf[(6, D)] = 6;

tf[(0, '"')] = 7;
tf[(7, '@')] = 7; # aqui o @ substitui ., aceita tudo 
tf[(7, '"')] = 8;

tf[(0, ' ')] = 23;
tf[(0, '\n')] = 23;
tf[(0, '\t')] = 23;

# Begin tf[(0, L)] = 9;
tf[(0, 'a')] = 9;
tf[(0, 'b')] = 9;
tf[(0, 'c')] = 9;
tf[(0, 'd')] = 9;
tf[(0, 'e')] = 9;
tf[(0, 'f')] = 9;
tf[(0, 'g')] = 9;
tf[(0, 'h')] = 9;
tf[(0, 'i')] = 9;
tf[(0, 'j')] = 9;
tf[(0, 'k')] = 9;
tf[(0, 'l')] = 9;
tf[(0, 'm')] = 9;
tf[(0, 'n')] = 9;
tf[(0, 'o')] = 9;
tf[(0, 'p')] = 9;
tf[(0, 'q')] = 9;
tf[(0, 'r')] = 9;
tf[(0, 's')] = 9;
tf[(0, 't')] = 9;
tf[(0, 'u')] = 9;
tf[(0, 'v')] = 9;
tf[(0, 'w')] = 9;
tf[(0, 'x')] = 9;
tf[(0, 'y')] = 9;
tf[(0, 'z')] = 9;
tf[(0, 'A')] = 9;
tf[(0, 'B')] = 9;
tf[(0, 'C')] = 9;
tf[(0, 'D')] = 9;
tf[(0, 'E')] = 9;
tf[(0, 'F')] = 9;
tf[(0, 'G')] = 9;
tf[(0, 'H')] = 9;
tf[(0, 'I')] = 9;
tf[(0, 'J')] = 9;
tf[(0, 'K')] = 9;
tf[(0, 'L')] = 9;
tf[(0, 'M')] = 9;
tf[(0, 'N')] = 9;
tf[(0, 'O')] = 9;
tf[(0, 'P')] = 9;
tf[(0, 'Q')] = 9;
tf[(0, 'R')] = 9;
tf[(0, 'S')] = 9;
tf[(0, 'T')] = 9;
tf[(0, 'U')] = 9;
tf[(0, 'V')] = 9;
tf[(0, 'W')] = 9;
tf[(0, 'X')] = 9;
tf[(0, 'Y')] = 9;
tf[(0, 'Z')] = 9;
# End tf[(0, L)] = 9;

# Begin tf[(9, L)] = 9;
tf[(9, 'a')] = 9;
tf[(9, 'b')] = 9;
tf[(9, 'c')] = 9;
tf[(9, 'd')] = 9;
tf[(9, 'e')] = 9;
tf[(9, 'f')] = 9;
tf[(9, 'g')] = 9;
tf[(9, 'h')] = 9;
tf[(9, 'i')] = 9;
tf[(9, 'j')] = 9;
tf[(9, 'k')] = 9;
tf[(9, 'l')] = 9;
tf[(9, 'm')] = 9;
tf[(9, 'n')] = 9;
tf[(9, 'o')] = 9;
tf[(9, 'p')] = 9;
tf[(9, 'q')] = 9;
tf[(9, 'r')] = 9;
tf[(9, 's')] = 9;
tf[(9, 't')] = 9;
tf[(9, 'u')] = 9;
tf[(9, 'v')] = 9;
tf[(9, 'w')] = 9;
tf[(9, 'x')] = 9;
tf[(9, 'y')] = 9;
tf[(9, 'z')] = 9;
tf[(9, 'A')] = 9;
tf[(9, 'B')] = 9;
tf[(9, 'C')] = 9;
tf[(9, 'D')] = 9;
tf[(9, 'E')] = 9;
tf[(9, 'F')] = 9;
tf[(9, 'G')] = 9;
tf[(9, 'H')] = 9;
tf[(9, 'I')] = 9;
tf[(9, 'J')] = 9;
tf[(9, 'K')] = 9;
tf[(9, 'L')] = 9;
tf[(9, 'M')] = 9;
tf[(9, 'N')] = 9;
tf[(9, 'O')] = 9;
tf[(9, 'P')] = 9;
tf[(9, 'Q')] = 9;
tf[(9, 'R')] = 9;
tf[(9, 'S')] = 9;
tf[(9, 'T')] = 9;
tf[(9, 'U')] = 9;
tf[(9, 'V')] = 9;
tf[(9, 'W')] = 9;
tf[(9, 'X')] = 9;
tf[(9, 'Y')] = 9;
tf[(9, 'Z')] = 9;
# End tf[(9, L)] = 9;



# Begin tf[(9, D)] = 9;
tf[(9, '0')] = 9;
tf[(9, '1')] = 9;
tf[(9, '2')] = 9;
tf[(9, '3')] = 9;
tf[(9, '4')] = 9;
tf[(9, '5')] = 9;
tf[(9, '6')] = 9;
tf[(9, '7')] = 9;
tf[(9, '8')] = 9;
tf[(9, '9')] = 9;
# End tf[(9, D)] = 9;

tf[(9, '_')] = 9;


tf[(0, '{')] = 10;
tf[(10, '@')] = 10; # aqui o @ substitui ., aceita tudo 
tf[(10, '}')] = 11;


tf[(0, '$')] = 12;

tf[(0, '>')] = 13;
tf[(13, '=')] = 14;

tf[(0, '<')] = 15;
tf[(15, '=')] = 14;
tf[(15, '>')] = 16;
tf[(15, '-')] = 17;

tf[(0, '=')] = 18;

# Begin tf[(0, OP)] = 19;
tf[(0, '+')] = 19;
tf[(0, '-')] = 19;
tf[(0, '*')] = 19;
tf[(0, '/')] = 19;
# End tf[(0, OP)] = 19;

tf[(0, '(')] = 20;
tf[(0, ')')] = 21;
tf[(0, ';')] = 22;

Definicao do alfabeto e estados

In [12]:
states = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23};
alphabet = {
    'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','y','x','z', 
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'X', 'Z',
    '0','1','2','3','4','5','6','7','8','9', 
    '+','-','*','/', 
    ' ','{','}','"',';','(',')','.', '>','<','='};
start_state = 0;
accept_states = {1, 3, 6, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23};

Inicializacao do automato

In [13]:
d = DFA(states, alphabet, tf, start_state, accept_states);

Definição do Analisador Léxico
----------------------------------------

In [14]:
class Scanner:
    dfa = None
    file = None
    rowList = None
    tokens = []
    errors = []
    i = 0
    j = 0
    end_of_analisys = False
    
    def __init__(self, dfa):  
        self.dfa = dfa
        self.file = open('fonte.alg')
        self.rowList = list(self.file)
        
    def restartAnalisys(self):
        self.end_of_analisys = False
        self.tokens = []
        self.errors = []
        self.i = 0 
        self.j = 0
        
    # Continue analisys until next token
    def getToken(self):
        if self.end_of_analisys:
            return Token("$", state_tokens[12], '-')
        self.dfa.go_to_initial_state();
        lexema = ""
        first = True
        while self.i < len(self.rowList):
            charList = list(self.rowList[self.i])
            # If is first iteration, get J from last stoped analisys
            if first:
                first = False
            else:
                # Else set j = 0 to first char on column
                self.j = 0
            while self.j < len(charList):
                
                didTransition = self.dfa.transition_to_state_with_input(charList[self.j]);
                # If is \n or \t or space or comment, ignore
                if(self.dfa.current_state is 23 or self.dfa.current_state is 11):
                    self.j = self.j + 1
                    self.dfa.go_to_initial_state()
                    lexema = ""
                    continue
                    
                if didTransition:
                    lexema = lexema + charList[self.j]
                    # Se for o ultimo lexema, caso seja erro, retornar eof, se não retornar o token e finalizar analise
                    if self.i is len(self.rowList)-1 and self.j is len(charList)-1:
                        # If is the last token
                        if self.dfa.in_accept_state():
                            #If has stopped in an accept state
                            token = Token(lexema, state_tokens[self.dfa.current_state], '-')
                            self.tokens.append(token)
                            self.dfa.go_to_initial_state()
                            lexema = ""
                            end_of_analisys = True
                            return token;
                        else:
                            self.errors.append([lexema, self.i+1, self.j+1])
                            print("!!! ERRO: Caracter {} não permitido. Na linha {} e coluna {}.".format(charList[self.j], self.i+1, self.j+1))
                            lexema = ""
                            self.dfa.go_to_initial_state()
                            return Token("$", state_tokens[12], '-')
                            
                else:
                    if self.dfa.in_accept_state():
                        token = Token(lexema, state_tokens[self.dfa.current_state], '-')
                        # Add to symbolTable if is ID 
                        if self.dfa.current_state is 9:
                            token = self.dfa.addToSymbolTable(lexema, token)
                        self.tokens.append(token)
                        self.dfa.go_to_initial_state()
                        lexema = ""
                        return token;
                    else: 
                        self.errors.append([lexema + charList[self.j], self.i+1, self.j+1])
                        print("!!! ERRO: Caracter {} não permitido. Na linha {} e coluna {}.".format(lexema + charList[self.j], self.i+1, self.j+1))
                        lexema = ""
                        self.dfa.go_to_initial_state()

                self.j = self.j + 1
            self.i = self.i + 1
            
            
    def doAnalisys(self):
        i = 0
        j = 0
        self.dfa.go_to_initial_state();
        tokens = []
        lexema = ""
        
        while i < len(self.rowList):
            charList = list(self.rowList[i])
            j = 0
            while j < len(charList):
                
                didTransition = self.dfa.transition_to_state_with_input(charList[j]);
    
                if(self.dfa.current_state is 23 or self.dfa.current_state is 11):
                    j = j + 1
                    self.dfa.go_to_initial_state()
                    lexema = ""
                    continue
                    
                if didTransition:
                    lexema = lexema + charList[j]
                    if i is len(self.rowList)-1 and j is len(charList)-1:
                        # If is the last token
                        if self.dfa.in_accept_state():
                            #If has stopped in an accept state
                            token = Token(lexema, state_tokens[self.dfa.current_state], '-')
                            print(token)
                            self.tokens.append(token)
                            self.dfa.go_to_initial_state()
                            lexema = ""
                        else:
                            self.errors.append([lexema, i+1, j+1])
                            print("!!! ERRO: Caracter {} não permitido. Na linha {} e coluna {}.".format(charList[j], i+1, j+1))
                            lexema = ""
                            self.dfa.go_to_initial_state()
                            
                else:
                    if self.dfa.in_accept_state():
                        token = Token(lexema, state_tokens[self.dfa.current_state], '-')
                        token = self.dfa.addToSymbolTable(lexema, token)
                        print(token)
                        self.tokens.append(token)
                        self.dfa.go_to_initial_state()
                        lexema = ""
                        continue
                    else: 
                        self.errors.append([lexema + charList[j], i+1, j+1])
                        print("!!! ERRO: Caracter {} não permitido. Na linha {} e coluna {}.".format(lexema + charList[j], i+1, j+1))
                        lexema = ""
                        self.dfa.go_to_initial_state()

                j = j + 1
            i = i + 1

In [15]:
d = DFA(states, alphabet, tf, start_state, accept_states);

In [16]:
scanner = Scanner(d)

In [17]:
scanner.restartAnalisys()

In [18]:
token = scanner.getToken()
print(token)
while token.token is not "EOF":
    token = scanner.getToken()
    print (token)

Lexema: inicio Token: inicio Tipo: -
Lexema: varinicio Token: varinicio Tipo: -
Lexema: A Token: id Tipo: -
Lexema: lit Token: lit Tipo: -
Lexema: ; Token: PTV Tipo: -
Lexema: B Token: id Tipo: -
Lexema: inteiro Token: inteiro Tipo: -
Lexema: ; Token: PTV Tipo: -
Lexema: D Token: id Tipo: -
Lexema: inteiro Token: inteiro Tipo: -
Lexema: ; Token: PTV Tipo: -
Lexema: C Token: id Tipo: -
Lexema: real Token: real Tipo: -
Lexema: ; Token: PTV Tipo: -
Lexema: varfim Token: varfim Tipo: -
Lexema: ; Token: PTV Tipo: -
Lexema: escreva Token: escreva Tipo: -
Lexema: "Digite B" Token: Literal Tipo: -
Lexema: ; Token: PTV Tipo: -
Lexema: leia Token: leia Tipo: -
Lexema: B Token: id Tipo: -
Lexema: ; Token: PTV Tipo: -
Lexema: escreva Token: escreva Tipo: -
Lexema: "Digite A:" Token: Literal Tipo: -
Lexema: ; Token: PTV Tipo: -
Lexema: leia Token: leia Tipo: -
Lexema: A Token: id Tipo: -
Lexema: ; Token: PTV Tipo: -
Lexema: se Token: se Tipo: -
Lexema: ( Token: AB_P Tipo: -
Lexema: B Token: id Tipo

In [ ]:
# scanner.doAnalisys()

In [ ]:
tokens_as_list = []
for token in scanner.tokens:
    tokens_as_list.append(token.getList())
print("\nTOKENS\n")
print("Foram reconhecidos {} tokens: \n".format(len(scanner.tokens)))
print(tabulate(tokens_as_list, headers=["LEXEMA","TOKEN", "TIPO"]))
if len(scanner.errors):
    print("\nCompilação finalizada com erros:\n")
    print("\nERROS\n")
    print(tabulate(scanner.errors, headers=["LEXEMA", "LINHA", "COLUNA"]))
else:
    print("\nCompilação finalizada.\n")